In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


# ファイル名を実際の訓練・テストデータファイル名に置き換えてください
train_data_rf = pd.read_csv('selected_train_set.csv')
test_data_rf = pd.read_csv('selected_test_set.csv')
index_col=['id']
# 特徴量（X）と目標変数（y）の設定
X_train_rf = train_data_rf.drop(columns=['PassengerId','Survived'])  # 'target_column'を目標変数の列名に置き換えてください
y_train_rf = train_data_rf['Survived']
X_test_rf = test_data_rf.drop(columns=['PassengerId','Survived'])
y_test_rf = test_data_rf['Survived']

# スケーリングとモデルのパイプライン設定
pipeline_rf = Pipeline([
    #('scaler', StandardScaler()),RFのみの時は、下のコードのみでよい
    ('model', RandomForestClassifier(n_estimators=200, random_state=42, max_depth=5, min_samples_split=5))
])


# 10回ループするためのリストを初期化
accuracies_rf = []  # 精度を格納するリスト
importances_list_rf = []  # 特徴量の重要度を格納するリスト
# 10回ループでモデルを作成
for i in range(10):
    # ランダムフォレストモデルの作成
    model_rf = RandomForestClassifier(n_estimators=200, random_state=i,max_depth=5, min_samples_split=5)
    
    # モデルの訓練
    model_rf.fit(X_train_rf, y_train_rf)
    
    # テストデータでの予測
    y_pred_rf = model_rf.predict(X_test_rf)
    
    # モデルの精度を計算
    accuracy_rf = accuracy_score(y_test_rf, y_pred_rf)
    accuracies_rf.append(accuracy_rf)
    
    # 特徴量の重要度を取得してリストに追加
    importances_rf = model_rf.feature_importances_
    importances_list_rf.append(importances_rf)

# 平均精度を計算
average_accuracy_rf = np.mean(accuracies_rf)
print(f'Average Accuracy over 10 runs: {average_accuracy_rf:.2f}')

# 特徴量の重要度の平均を計算
mean_importances_rf = np.mean(importances_list_rf, axis=0)
# 特徴量の名前と重要度の平均をデータフレームにまとめる
feature_importance_df_rf = pd.DataFrame({
    'Feature': X_train_rf.columns,
    'Importance': mean_importances_rf
})

# 重要度でソートし、上位20項目を抽出
top_13_features_rf = feature_importance_df_rf.sort_values(by='Importance', ascending=False).head(13)

# プロット
plt.barh(top_13_features_rf['Feature'], top_13_features_rf['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Average Feature Importance over 13 runs (Random Forest)')
plt.gca().invert_yaxis()  # 特徴量を重要度が高い順に表示
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc,confusion_matrix
# テストデータで予測確率を取得
y_scores_rf = model_rf.predict_proba(X_test_rf)[:, 1]

# ROC計算用のしきい値を決定（スコアの一意な値を使用）
thresholds_rf = np.sort(np.unique(y_scores_rf))

# 感度（TPR）と特異度（1 - FPR）を計算
tpr_list_rf = []
specificity_list_rf = []

for threshold in thresholds_rf:
    # スコアをしきい値で2値化
    y_pred_rf = (y_scores_rf >= threshold).astype(int)
    
    # 混同行列を取得
    tn_rf, fp_rf, fn_rf, tp_rf = confusion_matrix(y_test_rf, y_pred_rf).ravel()
    
    # 感度と特異度を計算
    tpr_rf = tp_rf / (tp_rf+ fn_rf) if (tp_rf + fn_rf) > 0 else 0  # 感度
    specificity_rf = tn_rf / (tn_rf + fp_rf) if (tn_rf + fp_rf) > 0 else 0  # 特異度
    
    # リストに保存
    tpr_list_rf.append(tpr_rf)
    specificity_list_rf.append(specificity_rf)

# AUCの計算
roc_auc_rf = auc(np.array(specificity_list_rf), tpr_list_rf)

# ROC曲線を描画
plt.figure(figsize=(8, 6))
plt.plot( np.array(specificity_list_rf), tpr_list_rf, label=f"ROC Curve (AUC = {roc_auc_rf:.2f})", color="blue")
plt.plot([1, 0], [0, 1], color="red", linestyle="--")  # 45度線
plt.xlim([1.0, 0.0])  # X軸を1から0に設定
plt.xlabel("Specificity ")
plt.ylabel("Sensitivity ")
plt.title("ROC Curve (Specificity vs Sensitivity)")
plt.legend(loc="lower left")
plt.grid()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 混同行列の計算
cm_rf = confusion_matrix(y_test_rf, y_pred_rf)

# 混同行列の要素を取得
tn_rf, fp_rf, fn_rf, tp_rf = cm_rf.ravel()  # 混同行列を分解

# 混同行列の表示
disp_rf = ConfusionMatrixDisplay(confusion_matrix=cm_rf, display_labels=["Negative", "Positive"])
disp_rf.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# 混同行列の要素を使って感度と特異度を計算
sensitivity_rf = tp_rf / (tp_rf + fn_rf)  # 感度
specificity_rf = tn_rf / (tn_rf + fp_rf)  # 特異度

print(f"Sensitivity: {sensitivity_rf:.2f}")
print(f"Specificity: {specificity_rf:.2f}")

In [ ]:
from sklearn.model_selection import learning_curve
# ロジスティック回帰モデルの定義
#model = LogisticRegression(random_state=42, max_iter=1000)

# 学習曲線の計算
train_sizes, train_scores, test_scores = learning_curve(
    model_rf, X_train_rf, y_train_rf, cv=5, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10)
)

# 各スコアの平均値と標準偏差を計算
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# 学習曲線のプロット
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores_mean, color='blue', label='Training Score')
plt.plot(train_sizes, test_scores_mean, color='green', label='Cross-validation Score')

# 標準偏差の影響をプロット
plt.fill_between(train_sizes, 
                 train_scores_mean - train_scores_std, 
                 train_scores_mean + train_scores_std, 
                 color='blue', alpha=0.2)
plt.fill_between(train_sizes, 
                 test_scores_mean - test_scores_std, 
                 test_scores_mean + test_scores_std, 
                 color='green', alpha=0.2)

plt.title('Learning Curve for Logistic Regression')
plt.xlabel('Training Set Size')
plt.ylabel('Score')
plt.legend(loc='best')
plt.grid()
plt.show()